# Import libraries

In [1]:
import os
import numpy as np
import time
from scipy.io import wavfile as wav
import sys

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import IPython.display as ipd

# Strumenti di classificazione
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Feature audio avanzate
import librosa
import librosa.display as lid
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


# TO DO: fix seed
# Load recordings

In [2]:
def load_recordings(paths = ["recordings"]):
    res = []
    for path in paths:
        print(f"Loading from {path}")    
        for f in tqdm(sorted(os.listdir(path))):
            if f.endswith('.wav'):
                # Carica file ed estraine le features
                audio, sample_rate = librosa.load(path + "/" + f)
                res.append(audio)

    return np.array(res)

In [3]:
def load_labels(paths = ["recordings"], label_type = "number"):

    labels = []
    
    for path in paths:
        for f in sorted(os.listdir(path)):
            if f.endswith('.wav'):
                if label_type.startswith("n"):
                    label = f.split('_')[0]
                else:
                    label = f.split('_')[1]
                labels.append(label)

    return labels

In [4]:
def compute_spectogram(audio, rate=8000, n_fft=1024, hop_length=160, n_mels=128, normalize=False):
    spectogram = librosa.feature.melspectrogram(y=np.array(audio),
                                                sr=rate,
                                                n_fft=n_fft,
                                                hop_length=hop_length,
                                                n_mels=n_mels)
    if normalize:
        spectogram = np.log10(10000*spectogram+1)
    return spectogram

In [5]:
recordings = load_recordings(paths=['recordings', 'output'])

Loading from recordings



Loading from output


Raw recordings have different lengths? Let's check it out:

In [6]:
min_y = min(map(np.shape, recordings))[0]
max_y = max(map(np.shape, recordings))[0]
print(min_y, max_y)

2784 50335


Yes! They vary a lot. For this reason we can add 0s at the beginning and at the end in order to uniform them

**TO DO: Another strategy may be to vary spectrogram params so that spectograms will have the same length**

In [7]:
def pad_zeros(recordings):
    min_y = min(map(np.shape, recordings))[0]
    max_y = max(map(np.shape, recordings))[0]
    res = []
    for rec in recordings:
        diff_in_y = max_y - rec.shape[0]
        if diff_in_y > 0:
            half_diff = int(diff_in_y/2)
            remaining_diff = diff_in_y-half_diff
            v = np.pad(rec,  ((half_diff,remaining_diff)), 'constant', constant_values=0)
            res.append(v)
        else:
            res.append(rec)
    return res

In [8]:
pad_recordings = pad_zeros(recordings)

What is the range now?

In [9]:
min_y = min(map(np.shape, pad_recordings))[0]
max_y = max(map(np.shape, pad_recordings))[0]
print(min_y, max_y)

50335 50335


We can now compute spectograms:

In [10]:
spects = [compute_spectogram(x) for x in pad_recordings]
spects = np.array(spects)

The procedure worked as expected! we can now move on to the prediction task

## Standard recordings
### Numbers

In [11]:
labels = load_labels(paths=['recordings', 'output'])

Split data in train and test

In [12]:
nsamples, nx, ny = spects.shape
spects_2d = spects.reshape((nsamples,nx*ny))
X_train, X_test, y_train, y_test = train_test_split(spects_2d, labels, test_size=0.2, random_state=1)

In [13]:
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")

In [14]:
%%time
clf1 = clf1.fit(X_train, y_train)

CPU times: user 3min 47s, sys: 1.7 s, total: 3min 49s
Wall time: 4min 1s


In [15]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.20      0.32        44
           1       0.18      0.78      0.29        45
           2       0.87      0.26      0.40        50
           3       0.39      0.58      0.47        45
           4       0.33      0.53      0.40        55
           5       0.77      0.40      0.52        43
           6       0.47      0.14      0.21        51
           7       0.83      0.32      0.46        47
           8       0.88      0.17      0.28        42
           9       0.79      0.39      0.53        38

    accuracy                           0.38       460
   macro avg       0.63      0.38      0.39       460
weighted avg       0.62      0.38      0.39       460

CPU times: user 36.1 s, sys: 262 ms, total: 36.4 s
Wall time: 37.8 s


### Normalize spectrograms

In [16]:
spects = [compute_spectogram(x, normalize=True) for x in pad_recordings]
spects = np.array(spects)

In [17]:
nsamples, nx, ny = spects.shape
spects_2d = spects.reshape((nsamples,nx*ny))
X_train, X_test, y_train, y_test = train_test_split(spects_2d, labels, test_size=0.2, random_state=1)

In [18]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 2min 18s, sys: 1.57 s, total: 2min 20s
Wall time: 2min 38s


In [19]:
%%time
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        44
           1       0.98      0.91      0.94        45
           2       0.81      0.96      0.88        50
           3       0.82      0.82      0.82        45
           4       1.00      1.00      1.00        55
           5       0.93      0.98      0.95        43
           6       0.83      0.84      0.83        51
           7       0.96      0.94      0.95        47
           8       0.94      0.76      0.84        42
           9       0.89      0.89      0.89        38

    accuracy                           0.91       460
   macro avg       0.91      0.91      0.91       460
weighted avg       0.91      0.91      0.91       460

CPU times: user 42.9 s, sys: 755 ms, total: 43.6 s
Wall time: 59.9 s


## CNNs

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kapp

### Normalized spectrograms

In [22]:
X_train, X_test, y_train, y_test = train_test_split(spects, labels, test_size=0.2, random_state=1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
input_shape = (X_train.shape[1], X_train.shape[2], 1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

### Paper architecture

In [23]:
def paper_architecture(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(4, 4), strides=(2,2), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(2,2)))
    model.add(Conv2D(64, kernel_size=(4, 4), strides=(2,2), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(10*num_classes, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5*num_classes, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [24]:
model = paper_architecture(10)

Instructions for updating:
Colocations handled automatically by placer.


In [25]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(nesterov=True),
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               652900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)              

In [26]:
%%time
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test));

Instructions for updating:
Use tf.cast instead.
Train on 1840 samples, validate on 460 samples
Epoch 1/10
1840/1840 [==============================] - 21s 11ms/step - loss: 2.3020 - accuracy: 0.1082 - val_loss: 2.2883 - val_accuracy: 0.1413
Epoch 2/10
1840/1840 [==============================] - 24s 13ms/step - loss: 2.2733 - accuracy: 0.1337 - val_loss: 2.2659 - val_accuracy: 0.2043
Epoch 3/10
1840/1840 [==============================] - 20s 11ms/step - loss: 2.2502 - accuracy: 0.1560 - val_loss: 2.2418 - val_accuracy: 0.1935
Epoch 4/10
1840/1840 [==============================] - 22s 12ms/step - loss: 2.2226 - accuracy: 0.1973 - val_loss: 2.2123 - val_accuracy: 0.1957
Epoch 5/10
1840/1840 [==============================] - 18s 10ms/step - loss: 2.1929 - accuracy: 0.2207 - val_loss: 2.1551 - val_accuracy: 0.2761
Epoch 6/10
1840/1840 [==============================] - 18s 10ms/step - loss: 2.1502 - accuracy: 0.2168 - val_loss: 2.1140 - val_accuracy: 0.2587
Epoch 7/10
1840/1840 [=======

In [27]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.30      0.43        44
           1       0.45      0.49      0.47        45
           2       0.66      0.76      0.70        50
           3       0.36      0.33      0.34        45
           4       0.95      0.65      0.77        55
           5       0.27      0.16      0.20        43
           6       0.74      0.63      0.68        51
           7       0.67      0.34      0.45        47
           8       0.71      0.52      0.60        42
           9       0.27      0.95      0.42        38

    accuracy                           0.52       460
   macro avg       0.58      0.51      0.51       460
weighted avg       0.60      0.52      0.52       460



### Normalization = False

In [ ]:
spects = [compute_spectogram(x) for x in pad_recordings]
spects = np.array(spects)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spects, labels, test_size=0.2, random_state=1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
input_shape = (X_train.shape[1], X_train.shape[2], 1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
model = paper_architecture(10)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(nesterov=True),
              metrics=['accuracy'])
print(model.summary())

In [ ]:
%%time
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test));

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

## Speakers

In [ ]:
labels = load_labels(paths=['recordings', 'output'], label_type="speakers")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
Y = enc.fit_transform(np.array(labels).reshape(-1, 1)).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spects, Y, test_size=0.2, random_state=1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [ ]:
model = paper_architecture(7)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(nesterov=True),
              metrics=['accuracy'])
print(model.summary())

In [ ]:
%%time
model.fit(X_train, y_train1,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test1));

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

Association between numbers and speakers:

In [402]:
enc.inverse_transform(np.array([0,0,0,0,0,0,0]).reshape(1, -1))

array([['alinda']], dtype='<U8')

In [406]:
enc.inverse_transform(np.array([0,1,0,0,0,0,0]).reshape(1, -1))

array([['gian']], dtype='<U8')

In [407]:
enc.inverse_transform(np.array([0,0,1,0,0,0,0]).reshape(1, -1))

array([['jackson']], dtype='<U8')

In [423]:
enc.inverse_transform(np.array([0,0,0,1,0,0,0]).reshape(1, -1))

array([['khaled']], dtype='<U8')

In [424]:
enc.inverse_transform(np.array([0,0,0,0,1,0,0]).reshape(1, -1))

array([['nicolas']], dtype='<U8')

In [425]:
enc.inverse_transform(np.array([0,0,0,0,0,1,0]).reshape(1, -1))

array([['theo']], dtype='<U8')

In [426]:
enc.inverse_transform(np.array([0,0,0,0,0,0,1]).reshape(1, -1))

array([['yweweler']], dtype='<U8')